<a href="https://colab.research.google.com/github/Tubhalooter/codeschool/blob/tss/TSS_JulyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the 30 days of code challenge!
___
Feel free to add your code files you have so far in the cells below.
If you need to install any packages for python, you can do below in a code cell by running:
```
!pip install package-name --upgrade --quiet
```
--upgrade is added so it installs the latest package versions and --quiet so it doesn't spam all over the output cell, it will only show error messages

In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 5.1MB/s 


In [ ]:

#ls code
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

# libraries imported^

# what page to use
url = 'https://spaceflightnow.com/launch-schedule/'

# request html text from page
html_text = requests.get(url).text

# lxml as parser(finds and then structures/makes sense of html text) to extract html code
html_S_code = BeautifulSoup(html_text, 'lxml')

# this uses the line above and we put in these arguments to then find the line of html code from the page

missionDate = html_S_code.find_all('div', class_="datename")
# to find mission date   # ^ find_all will find all instances of that type inside the brackets


missionData = html_S_code.find_all('div', class_="missiondata")
# to find info of mission

missionDesc = html_S_code.find_all('div', class_="missdescrip")
# to find description of mission

# the three variables above now hold all the lines of html code that match with their arguments


map = {}
id = -1  # for dictionary index counter inside for loop
num_mission = len(missionDate)

for i in range(num_mission):
    dateC = missionDate[i]
    dataC = missionData[i]
    descC = missionDesc[i]

    id += 1
    map[id] = {}

    lDate = dateC.text
    lData = dataC.text
    lDesc = descC.text

    # mapping to a location(id) in the dictionary below
    map[id]['Mission Date'] = lDate
    map[id]['Mission Data'] = lData
    map[id]['Mission Description'] = lDesc
    map[id]['mission num'] = []  # here we have created an empty list that is inside the dictionary under the category mission num, list in a dictionary
    # all of these^ are in the same location in the dictionary, just under different categories in same location

    # all of our scraped values are in a dictionary right now and need to be converted to a df
    # we need to convert to a list of lists to do this

    list = []  # list created

for id in range(len(missionDate)):  # the reason this works is because the parameter inside the len() points to a list,
    # len() of a list gives amount of values
    list.append([])  # appending the list

    list[id].append(str(map[id][
                            'Mission Date']))  # appending a certain value of the list with the string version of the value in (map[id]['missionData'])
    list[id].append(str(map[id]['Mission Data']))
    list[id].append(str(map[id]['Mission Description']))

events_to_df = pd.DataFrame(list, columns=['Mission Date', 'Mission Info', 'Mission Desc'])

MissionFile = 'MissionFiles.csv'
events_to_df.to_csv(MissionFile)

events_df = pd.read_csv('MissionFiles.csv')
events_df

cleaned_data = events_df.replace('\n', ' ', regex=True)# to replace data in a data frame, pretty self explanatory

cleaned_data

,Unnamed: 0,Mission Date,Mission Info,Mission Desc
0,0,July 21Proton • Nauka,Launch time: 1458 GMT (10:58 a.m. EDT) Launch ...,A Russian government Proton rocket will launch...
1,1,JulyFalcon 9 • Starlink,"Launch time: TBD Launch site: SLC-4E, Vandenbe...",A SpaceX Falcon 9 rocket will launch on the fi...
2,2,July 27Ariane 5 • Star One D2 & Eutelsat Quantum,"Launch window: TBD Launch site: ELA-3, Kourou,...","Arianespace will use an Ariane 5 ECA rocket, d..."
3,3,July 30Atlas 5 • CST-100 Starliner Orbital Fli...,Launch time: 1853 GMT (2:53 p.m. EDT) Launch s...,"A United Launch Alliance Atlas 5 rocket, desig..."
4,4,Aug. 2Soyuz • OneWeb 9,Launch time: TBD Launch site: Baikonur Cosmodr...,A Russian Soyuz rocket will launch 34 satellit...
5,5,Aug. 10Antares • NG-16,Launch time: 2155 GMT (5:55 p.m. EDT) Launch s...,A Northrop Grumman Antares rocket will launch ...
6,6,Mid-AugustVega • Pléiades Neo 4,"Launch time: TBD Launch site: ZLV, Kourou, Fre...","An Arianespace Vega rocket, designated VV19, w..."
7,7,Aug. 18Falcon 9 • SpaceX CRS 23,"Launch time: TBD Launch site: LC-39A, Kennedy ...",A SpaceX Falcon 9 rocket will launch a Dragon ...
8,8,Aug. 26Soyuz • OneWeb 10,Launch time: TBD Launch site: Baikonur Cosmodr...,A Russian Soyuz rocket will launch 34 satellit...
9,9,Early SeptemberAtlas 5 • STP-3,"Launch time: TBD Launch site: SLC-41, Cape Can...",A United Launch Alliance Atlas 5 rocket will l...


In [ ]:
launchSiteList=[]
for listloc in range(len(missionData)):
    launchSiteList.append([])
    find_launch_sites = str(events_df['Mission Info'][listloc]).split('\n')[1]
    launchSiteList[listloc].append(find_launch_sites)


LaunchSite_series = pd.Series(launchSiteList, name = 'Launch Sites')
LaunchS_File = 'Launch Sites List.csv'
LaunchSite_series.to_csv(LaunchS_File)

LaunchS_File_R = pd.read_csv('Launch Sites List.csv')
LaunchS_File_R

# this cell contains code to single out the launch site from our events_df dataframe
#it then puts it into a series of its own which will need to be split again to get rid of the prefix "Launch Site:"

,Unnamed: 0,Launch Sites
0,0,"['Launch site: Baikonur Cosmodrome, Kazakhstan']"
1,1,"['Launch site: SLC-4E, Vandenberg Space Force ..."
2,2,"['Launch site: ELA-3, Kourou, French Guiana']"
3,3,"['Launch site: SLC-41, Cape Canaveral Air Forc..."
4,4,"['Launch site: Baikonur Cosmodrome, Kazakhstan']"
5,5,"['Launch site: Pad 0A, Wallops Island, Virginia']"
6,6,"['Launch site: ZLV, Kourou, French Guiana']"
7,7,"['Launch site: LC-39A, Kennedy Space Center, F..."
8,8,"['Launch site: Baikonur Cosmodrome, Kazakhstan']"
9,9,"['Launch site: SLC-41, Cape Canaveral Space Fo..."


In [ ]:
gmaps = 'https://www.google.com/maps/search/'

for LSln in range (len(LaunchSite_series)):
    current_row = LaunchSite_series[LSln]
    cleaned_row = str(current_row).replace(' ','+')
    LSgmaps = (gmaps+str(cleaned_row))
    print(LSgmaps)



# my own solution to it,works now

https://www.google.com/maps/search/['Launch+site:+Baikonur+Cosmodrome,+Kazakhstan']
https://www.google.com/maps/search/['Launch+site:+SLC-4E,+Vandenberg+Space+Force+Base,+California']
https://www.google.com/maps/search/['Launch+site:+ELA-3,+Kourou,+French+Guiana']
https://www.google.com/maps/search/['Launch+site:+SLC-41,+Cape+Canaveral+Air+Force+Station,+Florida']
https://www.google.com/maps/search/['Launch+site:+Baikonur+Cosmodrome,+Kazakhstan']
https://www.google.com/maps/search/['Launch+site:+Pad+0A,+Wallops+Island,+Virginia']
https://www.google.com/maps/search/['Launch+site:+ZLV,+Kourou,+French+Guiana']
https://www.google.com/maps/search/['Launch+site:+LC-39A,+Kennedy+Space+Center,+Florida']
https://www.google.com/maps/search/['Launch+site:+Baikonur+Cosmodrome,+Kazakhstan']
https://www.google.com/maps/search/['Launch+site:+SLC-41,+Cape+Canaveral+Space+Force+Station,+Florida']
https://www.google.com/maps/search/['Launch+site:+LC-39A,+Kennedy+Space+Center,+Florida']
https://www.googl